# Data Cleaning and Preperation
---
DAT 512 Canisuis College <br>
Professor Paul Lambson<br>
<br>
### Learning Objectives
- Learn about multi-level indexes
- Learn how to join, merge and concat in pandas
- Reshape and learn pivoting and melting
<br>


### Sections
- [Hierarchical Indexing](#hierarhcical_indexing)
- [Reordeing and Sorting Levels](#reordering_and_sorting_levels)
- [Summary Statistics by Level](#summary_statistics_by_level)
- [Indexing with Columns](#indexing_with_columns)
- [Database Style Joins](#database_style_joins)
- [Merging on Index](#merging_on_index)
- [Concatenating Along an Axis](#concatenating_along_an_axis)
- [Combining Data with Overlap](#combining_data_with_overlap)
- [Reshaping and Pivoting](#reshaping_and_pivoting)
- [Pivoting Long to Wide Format](#pivoting_long_to_wide)
- [Pivoting Wide to Long Format](#pivoting_wide_to_long)

In [ ]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

<a id='hierarhcical_indexing'></a>
# Hierarchical Indexing

In [ ]:
# review "gaps" in multi-index
data = pd.Series(np.random.uniform(size=9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

In [ ]:
# not gaps but not showing the repeated data
data.index

In [ ]:
# able to use index notation
data["b"]

In [ ]:
data["b":"c"]

In [ ]:
data.loc[["b", "d"]]

In [ ]:
# can pull from inner level with the second index
data.loc[:, 2]

In [ ]:
# unstack can "pivot" the data
data.unstack()

In [ ]:
# unstack and stack are inverse
data.unstack().stack()

In [ ]:
# hierarchical level example on both axises
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"],
                              ["Green", "Red", "Green"]])
frame

In [ ]:
# able to rename indexes
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]
frame

In [ ]:
# method to show number of levels
frame.index.nlevels

In [ ]:
# can also select a group of columns
frame["Ohio"]

<a id='reordering_and_sorting_levels'></a>
# Reordering and Sorting Levels

In [ ]:
# can swap inner and outer levels
frame.swaplevel("key1", "key2")

In [ ]:
# specifcy level to sort on index
frame.sort_index(level=1)

In [ ]:
# when sorted on level = 0 the level isn't repeated in the view
frame.swaplevel(0, 1).sort_index(level=0)

<a id='summary_statistics_by_level'></a>
# Summary Statistics by Level

In [ ]:
# introducing split-apply-combine
frame.groupby(level="key2").sum()

In [ ]:
# works with columns when specified
frame.groupby(level="color", axis="columns").sum()

<a id='indexing_with_columns'></a>
# Indexing with a DataFrame's Columns

In [ ]:
# create frame wiht default range index
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                      "c": ["one", "one", "one", "two", "two",
                            "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})
frame

In [ ]:
# set multi-level index
frame2 = frame.set_index(["c", "d"])
frame2

In [ ]:
# retain columns
frame.set_index(["c", "d"], drop=False)

In [ ]:
# return index to columns and make a new index of range
frame2.reset_index()

# Combining and Merging Datasets

<a id='database_style_joins'></a>
# Database-Style DataFrame Joins

In [ ]:
# make dataframes to join
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})
df1


In [ ]:
df2

In [ ]:
# cross join, or cartesian join, not great
pd.merge(df1, df2)

In [ ]:
# pass a join key for an "inner" join
pd.merge(df1, df2, on="key")

In [ ]:
# left and right keys can be specified
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

In [ ]:
# SQL join types logic is honored
pd.merge(df1, df2, how="outer")

In [ ]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

In [ ]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                    "data1": pd.Series(range(6), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
                    "data2": pd.Series(range(5), dtype="Int64")})
df1

In [ ]:
df2

In [ ]:
# many to many makes many many outputs
pd.merge(df1, df2, on="key", how="left")

In [ ]:
# inner join still shows many many
pd.merge(df1, df2, how="inner")

In [ ]:
# to merge multiple keys pass a list of column names
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2": ["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3], dtype='Int64')})
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})
pd.merge(left, right, on=["key1", "key2"], how="outer")

In [ ]:
# output column names can be problematic
pd.merge(left, right, on="key1")

In [ ]:
# applying suffixes can identify origins
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

<a id='merging_on_index'></a>
# Merging on Index

In [ ]:
# mixed key, index and column
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])
left1

In [ ]:
right1

In [ ]:
right1
pd.merge(left1, right1, left_on="key", right_index=True)

In [ ]:
# overwrite default method from inner to outer
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

In [ ]:
# With hierarchically indexed data, things are more complicated, 
# as joining on index is equivalent to a multiple-key merge
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
                               "Nevada", "Nevada"],
                      "key2": [2000, 2001, 2002, 2001, 2002],
                      "data": pd.Series(range(5), dtype="Int64")})
righth_index = pd.MultiIndex.from_arrays(
    [
        ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)
righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64",
                                           index=righth_index),
                       "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64",
                                           index=righth_index)})
lefth

In [ ]:
righth

In [ ]:
# pass a list of keys that match the multi-level index
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)

In [ ]:
pd.merge(lefth, righth, left_on=["key1", "key2"],
         right_index=True, how="outer")

In [ ]:
# merging on index in both dataframes is possible
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"],
                     columns=["Ohio", "Nevada"]).astype("Int64")
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=["b", "c", "d", "e"],
                      columns=["Missouri", "Alabama"]).astype("Int64")
left2

In [ ]:
right2

In [ ]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

In [ ]:
# .join() method works to merge on index, simplified
left2.join(right2, how="outer")

In [ ]:
# left join by defauly, can speficy a join key on the passed DF
left1.join(right1, on="key")

In [ ]:
# simple index joins, a list of dataframes can be passed, fantastic
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=["a", "c", "e", "f"],
                       columns=["New York", "Oregon"])
another

In [ ]:
left2.join([right2, another])

In [ ]:
left2.join([right2, another], how="outer")

<a id='concatenating_along_an_axis'></a>
# Concatenating Along an Axis

In [ ]:
# numpy unlabeled array concats like a union
arr = np.arange(12).reshape((3, 4))
arr

In [ ]:
np.concatenate([arr, arr], axis=1)

In [ ]:
# 3 series with no index overlap
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

In [ ]:
# union all the series
s1
s2
s3
pd.concat([s1, s2, s3])

In [ ]:
#! union but new columns into a df
pd.concat([s1, s2, s3], axis="columns")

In [ ]:
# make an example series
s4 = pd.concat([s1, s3])
s4

In [ ]:
#overlapping index creates a join
pd.concat([s1, s4], axis="columns")

In [ ]:
# join comes with SQL logic
pd.concat([s1, s4], axis="columns", join="inner")

In [ ]:
# a mullti-level index can be created
result = pd.concat([s1, s1, s3], keys=["one", "two", "three"])
result

In [ ]:
result.unstack()

In [ ]:
# In the case of combining Series along axis="columns", the keys become the DataFrame column headers
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

In [ ]:
# same logic applies to dataframes
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"],
                   columns=["one", "two"])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"],
                   columns=["three", "four"])
df1

In [ ]:
df2

In [ ]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"])

In [ ]:
# pass a dictionary of objects instead of a list, the dictionary’s keys will be used for the keys
pd.concat({"level1": df1, "level2": df2}, axis="columns")

In [ ]:
# additional arguments
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"],
          names=["upper", "lower"])

<a id='combining_data_with_overlap'></a>
# Combining Data with Overlap

In [ ]:
#! ipython id=e4379b1ba6c74b9fbf6174fb4f2be35b
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
              index=["f", "e", "d", "c", "b", "a"])
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
              index=["a", "b", "c", "d", "e", "f"])
a

In [ ]:
b

In [ ]:
np.where(pd.isna(a), b, a)

In [ ]:
# use logic to combine
a.combine_first(b)

In [ ]:
# same logic exists on dataframes
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
                    "b": [np.nan, 2., np.nan, 6.],
                    "c": range(2, 18, 4)})
df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                    "b": [np.nan, 3., 4., 6., 8.]})
df1

In [ ]:
df2

In [ ]:
df1.combine_first(df2)

<a id='reshaping_and_pivoting'></a>
# Reshaping and Pivoting

In [ ]:
# dataframe to pivot
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"],
                    name="number"))
data

In [ ]:
#! stack pivots from columns in the data to rows
result = data.stack()
result

In [ ]:
# stack pivots from the rows into the columns
result.unstack()

In [ ]:
# can change the levels
result.unstack(level=0)

In [ ]:
result.unstack(level="state")

In [ ]:
# unstack my introduce missing data if all values aren't present
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")
data2 = pd.concat([s1, s2], keys=["one", "two"])
data2

In [ ]:
data2.unstack()

In [ ]:
data2.unstack().stack()

In [ ]:
data2.unstack().stack(dropna=False)

In [ ]:
# unstack in a DataFrame, the level unstacked becomes the lowest level in the result
df = pd.DataFrame({"left": result, "right": result + 5},
                  columns=pd.Index(["left", "right"], name="side"))
df

In [ ]:
df.unstack(level="state") 

In [ ]:
# As with unstack, when calling stack we can indicate the name of the axis to stack
df.unstack(level="state").stack(level="side")

<a id='pivoting_long_to_wide'></a>
# Pivoting Long to Wide Format

In [ ]:
#! ipython id=7691f5f737a44b39a67861744c98d2ae
data = pd.read_csv("examples/macrodata.csv")
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]
data.head()

In [ ]:
# recast index to a period rather than a point in time
periods = pd.PeriodIndex(year=data.pop("year"),
                         quarter=data.pop("quarter"),
                         name="date")
periods

In [ ]:
data.index = periods.to_timestamp("D")
data.head()

In [ ]:
# name the index
data = data.reindex(columns=["realgdp", "infl", "unemp"])
data.columns.name = "item"
data.head()

In [ ]:
# turn columns into rows with stack then reset index
long_data = (data.stack()
             .reset_index()
             .rename(columns={0: "value"}))

In [ ]:

long_data[:10]

In [ ]:
# use pivot to bring a colun to headers
pivoted = long_data.pivot(index="date", columns="item",
                          values="value")
pivoted.head()

In [ ]:
# add a second value to show multi-index
long_data["value2"] = np.random.standard_normal(len(long_data))
long_data[:10]

In [ ]:
# second value us upper index
pivoted = long_data.pivot(index="date", columns="item")
pivoted.head()

In [ ]:
pivoted["value"].head()

In [ ]:
# pivot is equivalent to creating a hierarchical index using set_index followed by a call to unstack
unstacked = long_data.set_index(["date", "item"]).unstack(level="item")
unstacked.head()

<a id='pivoting_wide_to_long'></a>
# Pivoting Wide to Long Format

In [ ]:
# example for melt
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                   "A": [1, 2, 3],
                   "B": [4, 5, 6],
                   "C": [7, 8, 9]})
df

In [ ]:
# indicate colum that is a group
melted = pd.melt(df, id_vars="key")
melted

In [ ]:
# pivot will return to original shape
reshaped = melted.pivot(index="key", columns="variable",
                        values="value")
reshaped

In [ ]:
# then reset the index
reshaped.reset_index()

In [ ]:
# specify a subset of columns to use as value columns
pd.melt(df, id_vars="key", value_vars=["A", "B"])

In [ ]:
# can be used without group identifiers
pd.melt(df, value_vars=["A", "B", "C"])

In [ ]:
pd.melt(df, value_vars=["key", "A", "B"])

# In Class Problems

In [ ]:
'''
    #1
    create 3 data frames from these urls
    {
        'products': 'https://raw.githubusercontent.com/sfrechette/adventureworks-neo4j/master/data/products.csv',
        'subcategories':'https://raw.githubusercontent.com/sfrechette/adventureworks-neo4j/master/data/productsubcategories.csv',
        'categories':'https://raw.githubusercontent.com/sfrechette/adventureworks-neo4j/master/data/productcategories.csv'
    }
'''
'''
    #2
    Evaluate all datafames and understand how they would join to each other
    Reindex each dataframe with its primary keyData Wrangling: Join, Combine, and Reshape
'''
'''
    #3
    Make a dataframe the includes data from all 3 data frames
'''
'''
    #4
    Perform some analysis on the combined data set
    
'''